<center>
    <img src="../imagens/logo_APL.png" width="300" alt="APL logo"  />
</center>

# Algoritmo: SVM (*Support Vector Machines*)

**Bem vindo!** Neste material você aprenderá o algoritmo  **SVM** (*Support Vector Machines*).  Usaremos a biblioteca  `Scikit-Learn`, desenvolvida para aplicação prática de algoritmos de *machine learning*. Ao final, espera-se que você seja capaz de entender e aplicar esse algoritmo.

<h2>Conteúdo:</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
<ul>
    <li> Introdução </li>
    <li> Entendendo e Carregando o Dataset </li> 
    <li> Análise Exploratória dos Dados </li>
    <li> Regressores (Matriz de entrada) e Variável Alvo (vetor de saídas) </li>  
    <li> Dividir o Dataset: Treinamento e Validação </li>   
    <li> Algoritmo de Machine Learning </li>  
    <li> Avaliação de Desempenho </li>  
</ul>
</div>

<hr>

## Introdução

Neste notebook, você usará SVM (Support Vector Machines) para construir e treinar um modelo usando registros de células humanas e classificar as células se as amostras são benignas ou malignas.

O SVM funciona mapeando os dados (**espaço das amostras**) para um espaço de dimensão superior (**espaço das características**) para que os pontos de dados possam ser categorizados a partir de um hiperplano. Isso se dá por meio da função kernel. 

A figura abaixo exemplifica esse mapeamento feito pela função kernel:

<center>
<img src="../imagens/SVM.png"  width="500">
</center>

Vamos começar importando as principais bibliotecas a serem utilizadas:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt

## Entendendo e Carregando o Dataset

Vamos carregar nosso dataset.
O exemplo é baseado em um conjunto de dados que está publicamente disponível no [UCI Machine Learning Repository](http://mlearn.ics.uci.edu/MLRepository.html). Esse conjunto de dados consiste em centenas de registros de amostras de células humanas, contendo diversas características. Os campos em cada registro são:

| Nome        | Descrição                             |
| ----------- | ---------------------------           |
| ID          | Identificação                         |
| Clump       | espessura do aglomerado               |
| UnifSize    | Uniformidade do tamanho da célula     |
| UnifShape   | Uniformidade do formato da célula     |
| MargAdh     | Adesão marginal                       |
| SingEpiSize | Tamanho de célula epitelial única     |
| BareNuc     | Núcleos puros                         |
| BlandChrom  | Cromatina branda                      |
| NormNucl    | Nucléolos normais                     |
| Mit         | Mitoses                               |
| Class       | Benigno ou Maligno                    |


Para fins de simplificação, estamos usando um conjunto de dados que possui um número relativamente pequeno de preditores ou regressores ou *features* (no presente contexto, esses termos são equivalentes) em cada registro.


In [ ]:
cell_df = pd.read_csv("https://raw.githubusercontent.com/APL-Data-Intelligence/AcelerAI/main/Curso_NEED/datasets/cellsample_dataset/cell_samples.csv")
cell_df.shape

Nosso dataset possui, portanto, 699 amostras e 11 características.

In [ ]:
cell_df.head()

## Análise Exploratória dos Dados

O campo **Class** contém o diagnóstico: se as amostras são benignas (valor = 2) ou malignas (valor = 4).

Vejamos a quantidade de cada uma dessas amostras segundo esse diagnóstico:

In [ ]:
cell_df['Class'].value_counts()

Vejamos a distribuição das classes (benigno ou maligno) com base nas features *CLump* e *UnifSize*:

In [ ]:
ax = cell_df[cell_df['Class'] == 4][0:50].plot(kind='scatter', x='Clump', y='UnifSize', color='DarkBlue', label='Maligno');
cell_df[cell_df['Class'] == 2][0:50].plot(kind='scatter', x='Clump', y='UnifSize', color='Yellow', label='Benigno', ax=ax);
plt.show()

## Separar as Colunas do Dataframe 
#### Regressores (Matriz de entrada) vs Variável Alvo (vetor de saídas).

Os regressores ou *features* são as variáveis explicativas do nosso dataset. Enquanto a variável alvo reflete a característica que desejamos classificar: em nosso dataset, se o câncer é benigno ou malígno.

Para essa finalidade, vamos criar um novo dataframe a partir do dataframe orignal, mas utilizando apenas as features desejadas.

In [ ]:
feature_df = cell_df[['Clump', 'UnifSize', 'UnifShape', 'MargAdh', 'SingEpiSize', 'BlandChrom', 'NormNucl', 'Mit']]
X = np.asarray(feature_df)
X[0:5]

Agora vamos criar o nosso dataframe que corresponde a variável alvo, ou seja, a variável que queremos fazer a predição.

In [ ]:
cell_df['Class'] = cell_df['Class'].astype('int')
y = np.asarray(cell_df['Class'])
y [0:5]

## Dividir o Dataset: Treinamento e Validação/Teste

Uma solução bastante conhecida para minimizar esse problema é separar a base de dados em dois subconjuntos, que são mutuamente exclusivos:  um para treinamento e outro para validação/teste. 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)

Vamos mostrar a dimensão desses conjuntos:

In [ ]:
print ('Conjunto de treino:', X_train.shape,  y_train.shape)
print ('Conjunto de teste:', X_test.shape,  y_test.shape)

Podemos observar que nosso **conjunto de treinamento** possui 559 amostras (ou 80%), enquanto o **conjunto de avaliação** 140 amostras (ou 20%), totalizando as 699 amostras originais. 

## Algoritmo de Machine Learning

#### **Passo 1:** Importando a componente do classificador. 
- No nosso exemplo vamos usar **Support Vector Machines**, na biblioteca `sklearn.svm`:

In [ ]:
from sklearn import svm

#### **Passo 2**: Instanciando o Estimador. 

In [ ]:
clf = svm.SVC()

#### **Passo 3**: Ajustar os parâmetros do modelo a partir dos dados

In [ ]:
clf.fit(X_train, y_train) 

#### **Passo 4:** Fazer a predição a partir de um novo comjunto de amostras:

In [ ]:
# Fazer previsões
y_pred = clf.predict(X_test)

## Avaliação de Desempenho

Vamos mostrar os valores de nosso dataset de validação e comparar com algumas predições feitas pelo nosso modelo:

In [ ]:
print("Valores reais:     ", y_test, "\n ")
print("Valores previstos: ", y_pred)

Comparando esses valores  podemos observar que nosso modelo acertou muitas de suas previsões. 

Para quantificar esa análise, vamos utilizar a métrica: **acurácia**, método `metrics.accuracy_score`. Essa métrica indica o percentual de classes foram corretamente classificadas.

In [ ]:
from sklearn import metrics
print("Precisão no conjunto de treino: ", 100*metrics.accuracy_score(y_train, clf.predict(X_train)), '%.')
print("Precisão no conjunto de teste: ", 100*metrics.accuracy_score(y_test, y_pred), '%.')

Podemos observar, portanto, que o nosso modelo obteve uma acurácia superior a 90%.

<hr>

## Direitos Autorais

[APL Data Intelligence](https://linktr.ee/APLdataintelligence)&#8482;  2021. Este notebook Python e seu código fonte estão liberados sob os termos da [Licença do MIT](https://bigdatauniversity.com/mit-license/).